# Trích xuất đặc trưng với TF-IDF

- Với bộ Corpus đã dọn `./data/cleaned_mhc.csv`, ta sử dụng phương pháp TF-IDF để trích xuất đặc trưng từ dữ liệu văn bản.

### Các bước thực hiện trích xuất đặc trưng từ TF-IDF và phân tích ma trận TF-IDF
- Thực hiện vector hóa các văn bản sử dụng `TfidfVectorizer` từ Sci-kit Learn, ta sẽ chọn tham số `max_features` phù hợp.
- Sau khi xác định được `max_features`, ta sẽ khảo sát về bản chất của ma trận TF-IDF thu được.
- Thực hiện phân tích về đồ tương đồng của các vector văn bản và vector token sử dụng Cosine Similarity.

### Công thức trọng số
Trọng số TF-IDF của  1 token $ i $ trong văn bản $ j $ được cho bởi công thức:

$$ w_{i, j} = tf_{i, j} \times \left( \log \left( \frac{N + 1}{df_i + 1} \right) + 1 \right) $$

Trong đó:
- $ N $ là tổng số văn bản trong Corpus.
- $ tf_{i, j}  $ là tần suất xuất hiện của token $ i $ trong văn bản $ j $.
- $ df_i $ là tần xuất có mặt của token $ i $ trong cả Corpus.

In [1]:
# Nhập thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Đọc bộ dữ liệu
corpus = pd.read_csv('data/cleaned_mhc.csv')

In [2]:
raw_shape = corpus.shape
print(f"Original shape of corpus: {raw_shape}")

max_features_list = [100, 200, 500, 1000, 2000, 3000, 3500, 4500, 6000]
tfidf_vectorizers = [TfidfVectorizer(max_features=mf) for mf in max_features_list]

tfidf_matrices = {
    f"tfidf_matrix_{max_features}": vectorizer.fit_transform(corpus['text'])
    for max_features, vectorizer in zip(max_features_list, tfidf_vectorizers)
}

for name, matrix in tfidf_matrices.items():
    print(f"Shape of {name}: {matrix.shape}")

Original shape of corpus: (23240, 2)
Shape of tfidf_matrix_100: (23240, 100)
Shape of tfidf_matrix_200: (23240, 200)
Shape of tfidf_matrix_500: (23240, 500)
Shape of tfidf_matrix_1000: (23240, 1000)
Shape of tfidf_matrix_2000: (23240, 2000)
Shape of tfidf_matrix_3000: (23240, 3000)
Shape of tfidf_matrix_3500: (23240, 3500)
Shape of tfidf_matrix_4500: (23240, 4500)
Shape of tfidf_matrix_6000: (23240, 6000)


In [3]:
least_important_tokens = {}

for i, (tfidf_matrix_name, tfidf_matrix) in enumerate(tfidf_matrices.items()):
    tfidf_vectorizer = tfidf_vectorizers[i]
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    mean_tfidf_scores = tfidf_matrix.mean(axis=0).A1
    
    tfidf_df = pd.DataFrame({
        'token': feature_names,
        'mean_tfidf': mean_tfidf_scores
    })
    
    least_important = tfidf_df.nsmallest(20, 'mean_tfidf')
    least_important_tokens[tfidf_matrix_name] = least_important['token'].values

least_important_df = pd.DataFrame(dict(least_important_tokens))
print(least_important_df.to_string(index=False))

tfidf_matrix_100 tfidf_matrix_200 tfidf_matrix_500 tfidf_matrix_1000 tfidf_matrix_2000 tfidf_matrix_3000 tfidf_matrix_3500 tfidf_matrix_4500 tfidf_matrix_6000
            told          college            moved         treatment            income           maddock           maddock        clickclick           friesmy
           tried       understand            water           current           officer             clack               haa               hoo           clinker
           happy            least       eventually            abused           manager             brice             clack           maddock           scutter
          around             used          control          homeless           appears          intended             ameno               haa        clickclick
           month             kind              met           mention        threatened         narrative             brice               dor               haa
          reason              yet          out

### Quyết định giới hạn vốn từ vựng
- Qua việc phân tích các ma trận TF-IDF với các `max_features` khác nhau, ta thấy rằng đuôi của những ma trận này, thường là những token "ít quan trọng" đối với dữ liệu.
- Tuy nhiên, ta cần đánh đổi giữa độ rộng của vốn từ vựng và việc thu nhập lượng nhiễu nhỏ nhất có thể vào các đặc trưng do trong quá trình đánh giá, TF-IDF sẽ cho tất cả các token không có trong vốn từ vựng của nó trọng số bằng 0.
- Ta thấy rằng với `max_features` bằng 3500 hay vốn từ vựng bằng 3500, TF-IDF vẫn có thể trích xuất các token có nghĩa mà có thể là ta không muốn bỏ qua, từ 4500 trở đi, các token được coi là "ít quan trọng" phần lớn có vẻ là các từ vô nghĩa hoặc tên riêng, vì vậy, ta sẽ chọn TF-IDF với `max_features` = 3500 làm thước đo tiêu chuẩn cho các quá trình đánh giá sau.

### Công thức Cosine Similarity
Với 2 vector $ u $ và $ v $, Cos của góc giữa chúng được gọi là Cosine Similarity:
$ Sim_c(u, v) := \cos(\theta) = \frac{u \cdot v}{|| u || \cdot || v ||}$

Với 2 vector được chuẩn hóa $ || u || = 1 $ và  $ || v || = 1 $, Cosine Similarity giữa chúng đơn giản là tích vô hướng:
$ \cos(\theta) = u \cdot v $

In [4]:
from scipy.sparse import csr_matrix

tfidf = TfidfVectorizer(max_features=3500)
tfidf_matrix = tfidf.fit_transform(corpus['text'])

print(f"Shape of TF-IDF matrix: {tfidf_matrix.shape}")

np.random.seed(42)
random_indices = np.random.choice(tfidf_matrix.shape[0], 10, replace=False)

print(f"{'Doc. Index':<12} {'Zeros':<10} {'Non-Zero':<10} {'Magnitude':<10}")
print("-" * 42)
for i in random_indices:
    doc = tfidf_matrix[i]
    zero_count = doc.shape[1] - doc.count_nonzero()
    non_zero_count = doc.count_nonzero()
    magnitude = np.sqrt(doc.multiply(doc).sum())
    print(f"{i:<12} {zero_count:<10} {non_zero_count:<10} {magnitude:<10.4f}")

Shape of TF-IDF matrix: (23240, 3500)
Doc. Index   Zeros      Non-Zero   Magnitude 
------------------------------------------
11456        3473       27         1.0000    
1288         3488       12         1.0000    
5535         3489       11         1.0000    
12657        3492       8          1.0000    
8603         3482       18         1.0000    
8828         3449       51         1.0000    
16011        3490       10         1.0000    
7779         3485       15         1.0000    
8779         3466       34         1.0000    
11941        3420       80         1.0000    


### Dữ liệu ma trận thưa
- Ta có thể thấy rằng bản chất của các ma trận TF-IDF là chúng rất lớn và thường rất thưa, như có thể quan sát ở trên.
- Chúng ta sẽ cần sử dụng những phương pháp phù hợp hơn khi làm việc với dữ liệu thưa nói riêng và dữ liệu văn bản nói chung.
- Ngoài ra, `TfidfVectorizer` mặc định sẽ chuẩn hóa các vector văn bản, khiến chúng có độ lớn bằng 1, điều này giúp giảm thiểu sự nhạy cảm của phương pháp đối với độ dài của văn bản khác nhau.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

corpus['token_count'] = corpus['text'].apply(lambda x: len(x.split()))
filtered_corpus = corpus[(corpus['token_count'] >= 15) & (corpus['token_count'] <= 25)]

np.random.seed(0)
class_0_docs = filtered_corpus[filtered_corpus['label'] == 0].sample(2, random_state=0).index
class_1_docs = filtered_corpus[filtered_corpus['label'] == 1].sample(2, random_state=0).index
selected_docs = list(class_0_docs) + list(class_1_docs)

cosine_sim_matrix = cosine_similarity(tfidf_matrix)

for doc_id in selected_docs:
    full_text = corpus.loc[doc_id, 'text']
    print(f"\nDocument Index {doc_id} (Class {corpus.loc[doc_id, 'label']}):\n{full_text}\n")

    sim_scores = list(enumerate(cosine_sim_matrix[doc_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    
    print(f"{'Similar Doc':<12} {'Class':<6} {'Similarity':<10} {'Text':<50}")
    print("-" * 80)
    for sim_doc_id, score in sim_scores:
        sim_doc_class = corpus.loc[sim_doc_id, 'label']
        truncated_text = corpus.loc[sim_doc_id, 'text'][:50] + "..."
        print(f"{sim_doc_id:<12} {sim_doc_class:<6} {score:<10.4f} {truncated_text:<50}")


Document Index 15464 (Class 0):
guess show dude love girl yes love much dude wanna make love girl sorry cant im ready yet dude ready girl abortion hour ago

Similar Doc  Class  Similarity Text                                              
--------------------------------------------------------------------------------
19288        0      0.4928     anyone talk girl problem thats dude think know eve...
11111        0      0.4265     normalize calling guy queen girl bro dude day got ...
7077         0      0.4029     see one dude skirt ima make convention fucking tod...
16470        0      0.3710     care im dude want love mean hard girl think guy wa...
10582        0      0.3667     school picture taken look like female version dude...

Document Index 16482 (Class 0):
inspiring movie laughed cried felt love true give hope miracle happen great cast ellen samantha old actress showtime must see movie

Similar Doc  Class  Similarity Text                                              
------

### Sử dụng Cosine Similarity trên các Vector văn bản

- Cosine Similarity đo góc giữa các vector, vì vậy với các vector văn bản, ta có thể hiểu rằng những kết quả này thể hiện độ "tương đồng" dựa trên "độ quan trọng" của các token xuất hiện trong văn bản.
- Thậm chí ta thấy các các vector được cho là "tương đồng" với các văn bản được chọn đôi khi cũng có cùng lớp với nhau.

In [6]:
tokens = tfidf.get_feature_names_out()

token_similarity_matrix = cosine_similarity(tfidf_matrix.T)

token_sim_df = pd.DataFrame(token_similarity_matrix, index=tokens, columns=tokens)

tokens_of_interest = ['suicidal', 'depression', 'suicide', 'kill', 'myself', 'die',
                         'pain', 'sad', 'help', 'sorry', 'anxiety', 'therapy',
                         'suffering', 'pill', 'redflag',
                         'movie', 'film', 'character', 'story', 'actor', 'performance', 'show',
                         'plot', 'acting']

for token in tokens_of_interest:
    if token in tokens:
        print(f"\nTop 7 terms with highest co-occurrence score for '{token}':")
        co_occurrences = token_sim_df[token].sort_values(ascending=False)[1:8]
        for term, score in co_occurrences.items():
            print(f"Token: {term:<15} Co-occurrence Score: {score:.4f}")
    else:
        print(f"\nToken '{token}' not found in the vocabulary.")


Top 7 terms with highest co-occurrence score for 'suicidal':
Token: thought         Co-occurrence Score: 0.3086
Token: know            Co-occurrence Score: 0.1934
Token: im              Co-occurrence Score: 0.1856
Token: ideation        Co-occurrence Score: 0.1846
Token: feel            Co-occurrence Score: 0.1770
Token: feeling         Co-occurrence Score: 0.1750
Token: ive             Co-occurrence Score: 0.1694

Top 7 terms with highest co-occurrence score for 'depression':
Token: anxiety         Co-occurrence Score: 0.2359
Token: feel            Co-occurrence Score: 0.2037
Token: year            Co-occurrence Score: 0.1991
Token: life            Co-occurrence Score: 0.1908
Token: im              Co-occurrence Score: 0.1825
Token: ive             Co-occurrence Score: 0.1767
Token: like            Co-occurrence Score: 0.1718

Top 7 terms with highest co-occurrence score for 'suicide':
Token: cannot          Co-occurrence Score: 0.1240
Token: hotline         Co-occurrence Score: 0.08

### Sử dụng Cosine Similarity trên các Vector token

- Với vector token, có sự khác biệt so với vector văn bản.
- Do bản chất TF-IDF không hiểu được ngữ nghĩa sâu xa, nên những kết quả trên có thể hiểu là độ "tương đồng dựa trên độ quan trọng trên cả Corpus" của mỗi token.
- Chính vì vậy, những vector token có góc gần nhau chưa chắc đã là đồng nghĩa hay giống nhau mà có thể đơn giản chỉ là thường xuất hiện cùng nhau.
- Điều này đồng thời cho thấy hạn chế của phương pháp khi so với các kĩ thuật Word Embedding.